In [6]:
from composio_openai import ComposioToolSet, App, Action
from dotenv import load_dotenv
import os

load_dotenv()

True

In [7]:
google_api_key=os.getenv('google_api_key')

In [10]:
llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))


In [4]:
composio_toolset = ComposioToolSet(api_key="ui9jbd366fmtu1y6ekotd")
tools = composio_toolset.get_tools(apps=[App.GOOGLECALENDAR])



[2025-03-29 13:56:49,956][INFO] Actions cache is outdated, refreshing cache...
c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\mcp-testing\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions

Give Feedback / Get Help:
    On GitHub: https://github.com/ComposioHQ/composio/issues/new
    On Discord: https://dub.composio.dev/discord
    On Email: tech@composio.dev
    Talk to us on Intercom: https://composio.dev
    Book a call with us: https://composio.dev/redirect?url=https://calendly.com/composiohq/support?utm_source=py-sdk-logs&utm_campaign=calendly
If you need to debug this error, set `COMPOSIO_LOGGING_LEVEL=debug`.
  warnings.warn(


In [ ]:
agent = Agent(llm, tools=tools)


TypeError: [{'type': 'function', 'function': {'name': 'GOOGLECALENDAR_FIND_EVENT', 'description': 'Find events in a google calendar based on a search query.', 'parameters': {'properties': {'calendar_id': {'default': 'primary', 'description': "Identifier of the Google Calendar. Use 'primary' for the currently logged in user's primary calendar. Please provide a value of type string.", 'title': 'Calendar Id', 'type': 'string'}, 'query': {'default': None, 'description': "Search term to find events that match these terms in the event's summary, description, location, attendee's displayName, attendee's email, organizer's displayName, organizer's email, etc if needed. Please provide a value of type string.", 'title': 'Query', 'type': 'string'}, 'max_results': {'default': 10, 'description': 'Maximum number of events returned on one result page. The page size can never be larger than 2500 events. The default value is 10. Please provide a value of type integer.', 'title': 'Max Results', 'type': 